# Introduction

We tackle the problem of OCR post processing. In OCR, we map the image form of the document into the text domain. This is done first using an CNN+LSTM+CTC model, in our case based on tesseract. Since this output maps only image to text, we need something on top to validate and correct language semantics.

The idea is to build a language model, that takes the OCRed text and corrects it based on language knowledge. The langauge model could be:
- Char level: the aim is to capture the word morphology. In which case it's like a spelling correction system.
- Word level: the aim is to capture the sentence semnatics. But such systems suffer from the OOV problem.
- Fusion: to capture semantics and morphology language rules. The output has to be at char level, to avoid the OOV. However, the input can be char, word or both.

The fusion model target is to learn:

    p(char | char_context, word_context)

In this workbook we use seq2seq vanilla Keras implementation, adapted from the lstm_seq2seq example on Eng-Fra translation task. The adaptation involves:

- Adapt to spelling correction, on char level
- Pre-train on a noisy, medical sentences
- Fine tune a residual, to correct the mistakes of tesseract 
- Limit the input and output sequence lengths
- Enusre teacher forcing auto regressive model in the decoder
- Limit the padding per batch
- Learning rate schedule
- Bi-directional LSTM Encoder
- Bi-directional GRU Encoder


# Imports

In [1]:
from __future__ import print_function
import tensorflow as tf
import keras.backend as K
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate, GRU
from keras import optimizers
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


# Utility functions

In [2]:
# Limit gpu allocation. allow_growth, or gpu_fraction
def gpu_alloc():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.Session(config=config))

In [3]:
gpu_alloc()

In [4]:
def calculate_WER_sent(gt, pred):
    '''
    calculate_WER('calculating wer between two sentences', 'calculate wer between two sentences')
    '''
    gt_words = gt.lower().split(' ')
    pred_words = pred.lower().split(' ')
    d = np.zeros(((len(gt_words) + 1), (len(pred_words) + 1)), dtype=np.uint8)
    # d = d.reshape((len(gt_words)+1, len(pred_words)+1))

    # Initializing error matrix
    for i in range(len(gt_words) + 1):
        for j in range(len(pred_words) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(gt_words) + 1):
        for j in range(1, len(pred_words) + 1):
            if gt_words[i - 1] == pred_words[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)
    return d[len(gt_words)][len(pred_words)]

In [5]:
def calculate_WER(gt, pred):
    '''

    :param gt: list of sentences of the ground truth
    :param pred: list of sentences of the predictions
    both lists must have the same length
    :return: accumulated WER
    '''
#    assert len(gt) == len(pred)
    WER = 0
    nb_w = 0
    for i in range(len(gt)):
        #print(gt[i])
        #print(pred[i])
        WER += calculate_WER_sent(gt[i], pred[i])
        nb_w += len(gt[i])

    return WER / nb_w

In [6]:
def load_data_with_gt(file_name, num_samples, max_sent_len, min_sent_len):
    '''Load data from txt file, with each line has: <TXT><TAB><GT>. The  target to the decoder muxt have \t as the start trigger and \n as the stop trigger.'''
    cnt = 0  
    input_texts = []
    gt_texts = []
    target_texts = []
    for row in open(file_name, encoding='utf8'):
        if cnt < num_samples :
            #print(row)
            sents = row.split("\t")
            input_text = sents[0]
            
            target_text = '\t' + sents[1] + '\n'
            if len(input_text) > min_sent_len and len(input_text) < max_sent_len and len(target_text) > min_sent_len and len(target_text) < max_sent_len:
                cnt += 1
                
                input_texts.append(input_text)
                target_texts.append(target_text)
                gt_texts.append(sents[1])
    return input_texts, target_texts, gt_texts

In [7]:
def load_data(file_name, num_samples, max_sent_len, min_sent_len):
    '''Load data from txt file, with each line has: <TXT><TAB><GT>. The  target to the decoder muxt have \t as the start trigger and \n as the stop trigger.'''
    cnt = 0  
    input_texts = []   
    
    #for row in open(file_name, encoding='utf8'):
    for row in open(file_name):
        if cnt < num_samples :            
            input_text = row           
            if len(input_text) > min_sent_len and len(input_text) < max_sent_len:
                cnt += 1                
                input_texts.append(input_text)
    return input_texts

In [8]:
def vectorize_data(input_texts, max_encoder_seq_length, num_encoder_tokens, vocab_to_int):
    '''Prepares the input text and targets into the proper seq2seq numpy arrays'''
    encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

    for i, input_text in enumerate(input_texts):
        for t, char in enumerate(input_text):
            # c0..cn
            encoder_input_data[i, t, vocab_to_int[char]] = 1.
                
    return encoder_input_data

In [9]:
def decode_sequence(input_seq, encoder_model, decoder_model, num_decoder_tokens, int_to_vocab):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, vocab_to_int['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = int_to_vocab[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


# Load model params

In [10]:
vocab_file = 'vocab.npz'
model_file = 'best_model.hdf5'
encoder_model_file = 'encoder_model.hdf5'
decoder_model_file = 'decoder_model.hdf5'

In [11]:
vocab = np.load(file=vocab_file)
vocab_to_int = vocab['vocab_to_int'].item()
int_to_vocab = vocab['int_to_vocab'].item()
max_sent_len = vocab['max_sent_len']
min_sent_len = vocab['min_sent_len']
print(max_sent_len)
print(min_sent_len)


100
4


# Load data

In [12]:
data_path = '../../dat/'

In [13]:
num_samples = 1000000
tess_correction_data = os.path.join(data_path, 'all_ocr_data_2.txt')
input_texts = load_data(tess_correction_data, num_samples, max_sent_len, min_sent_len)

In [14]:
input_characters = sorted(list(vocab_to_int))
num_decoder_tokens = num_encoder_tokens = len(input_characters) #int(encoder_model.layers[0].input.shape[2])
max_encoder_seq_length = max_decoder_seq_length = max_sent_len - 1#max([len(txt) for txt in input_texts])


In [15]:
max_encoder_seq_length

99

In [16]:
#model.load_weights(model_file)
from keras.models import load_model
model = load_model(model_file)

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None, 121)    0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, None, 121)    0                                            
__________________________________________________________________________________________________
lstm_7 (LSTM)                   [(None, 256), (None, 387072      input_7[0][0]                    
__________________________________________________________________________________________________
lstm_8 (LSTM)                   [(None, None, 256),  387072      input_8[0][0]                    
                                                                 lstm_7[0][1]                     
          

In [18]:
encoder_model = load_model(encoder_model_file)
decoder_model = load_model(decoder_model_file)

/root/anaconda2/lib/python2.7/site-packages/keras/models.py:252: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [19]:
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, None, 121)         0         
_________________________________________________________________
lstm_7 (LSTM)                [(None, 256), (None, 256) 387072    
Total params: 387,072
Trainable params: 387,072
Non-trainable params: 0
_________________________________________________________________


In [ ]:
decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, None, 121)    0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 256)          0                                            
__________________________________________________________________________________________________
lstm_8 (LSTM)                   [(None, None, 256),  387072      input_8[0][0]                    
                                                                 input_9[0][0]                    
          

In [ ]:


encoder_input_data = vectorize_data(input_texts=input_texts, max_encoder_seq_length=max_encoder_seq_length, num_encoder_tokens=num_encoder_tokens, vocab_to_int=vocab_to_int)

# Sample output from train data
decoded_sentences = []
target_texts_ =  []
for seq_index in range(len(input_texts)):
    # Take one sequence (part of the training set)
    # for trying out decoding.

    input_seq = encoder_input_data[seq_index: seq_index + 1]
    
    decoded_sentence = decode_sequence(input_seq, encoder_model, decoder_model, num_decoder_tokens, int_to_vocab)
    
    print('-')
    print('Input sentence:', input_texts[seq_index])
    
    print('Decoded sentence:', decoded_sentence)   
    decoded_sentences.append(decoded_sentence)
    
    
WER_spell_correction = calculate_WER(target_texts_, decoded_sentences)
print('WER_spell_correction |TEST= ', WER_spell_correction)

WER_OCR = calculate_WER(target_texts_, input_texts)
print('WER_OCR |TEST= ', WER_OCR)

-
Input sentence: Claim Type: VB Accident - Accidental Injury	Claim Type: VB Accident - Accidental Injury

Decoded sentence: Call toll-free Monday through (mm/dd/yy)

-
Input sentence: Pol inyhold elm-Chm er [11 form arlon	Policyholder/Owner Information

Decoded sentence: Patient Name:

-
Input sentence: First Name:	First Name:

Decoded sentence: Provider Signed Information

-
Input sentence: Middle Nameﬂnitial:	Middle Name/Initial:

Decoded sentence: Provider Signed Information

-
Input sentence: Last Name:	Last Name:

Decoded sentence: Call toll-free Monday through (mm/dd/yy)

-
Input sentence: Social S ecurity Number:	Social Security Number:

Decoded sentence: Patient Name:

-
Input sentence: Birth Date:	Birth Date:

Decoded sentence: Provider Signed Information

-
Input sentence: Gender:	Gender:

Decoded sentence: Patient Name:

-
Input sentence: Language Preference:	Language Preference:

Decoded sentence: Catient Name:

-
Input sentence: Address Line 1:	Address Line 1:

Decoded se

-
Input sentence: Electronic Submission	Electronic Submission

Decoded sentence: Employee Poller Information

-
Input sentence: ClairnEvent Identifier:	Claim Event Identifier:

Decoded sentence: Call toll-free Monday through (mm/dd/yy)

-
Input sentence: Submission Date:	Submission Date:

Decoded sentence: Current Name:

-
Input sentence: Electronically Signed Indicator: Yes	Electronically Signed Indicator: Yes

Decoded sentence: Employee Poller Information

-
Input sentence: Fraud Statements Reviewed and Electronically	Fraud Statements Reviewed and Electronically

Decoded sentence: Provider Signed Information

-
Input sentence: Signed Date:	Signed Date:

Decoded sentence: Patient Name:

-
Input sentence: unumw	unum

Decoded sentence: Cumber Signed Information

-
Input sentence: C O O The Benefits Center	The Benefits Center

Decoded sentence: Employee Patient Name:

-
Input sentence: (Not for FMLA Requests)	(Not for FMLA Requests)

Decoded sentence: Provider Signed Information

-
Input

-
Input sentence: Claim E‘iﬁ'lH Information	Claim Event Information

Decoded sentence: Call toll-free Monday through (mm/dd/yy)

-
Input sentence: Accident Work Related: N o	Accident Work Related: No

Decoded sentence: Accident Information

-
Input sentence: Time ofAccident:	Time of Accident:

Decoded sentence: Current Name:

-
Input sentence: Accident Date:	Accident Date:

Decoded sentence: Accident Information

-
Input sentence: Surg 91'3'lnf01'Iilation No	Surgery Information

Decoded sentence: Surgery Signed Information

-
Input sentence: Is Surgery Required:	Is Surgery Required: No

Decoded sentence: Employee Poller Information

-
Input sentence: \Ipdira] Pl'mitlel' In formation 7 Physician	Medical Provider Information - Physician

Decoded sentence: Employee Patient Name:

-
Input sentence: Medical Provider Specialty. Orthopedic Surgeon	Medical Provider Specialty: Orthopedic Surgeon

Decoded sentence: Provider Signed Information

-
Input sentence: Medical Provider Roles:  Treating	

-
Input sentence: Total Employee Semi-Montth Payroll Deduction:	Total Employee Semi-Monthly Payroll Deduction:

Decoded sentence: Call toll-free Monday through (mm/dd/yy)

-
Input sentence: BLACK HILLS ORTHOPEDIC & SPINE CENTER, PC -. m	BLACK HILLS ORTHOPEDIC & CENTER, PC

Decoded sentence: Address Line 1:

-
Input sentence: ‘3'" 16466-384N	16466-384N

Decoded sentence: Current Name:

-
Input sentence: ' LAST PMTZ	LAST PMT:

Decoded sentence: Procedure Provider Information

-
Input sentence: 254895	254895

Decoded sentence: Address Line 1:

-
Input sentence: AMOUNT:	AMOUNT: 

Decoded sentence: Address Line 1:

-
Input sentence: DUE DATE :	DUE DATE:

Decoded sentence: Accident Information

-
Input sentence: PAGE: 1 Of 1	PAGE 1 of 1

Decoded sentence: Address Line 1:

-
Input sentence: STATEMENT *1	STATEMENT

Decoded sentence: Address Line 1:

-
Input sentence: Date	Date

Decoded sentence: Cate Dischork of Sectide the to the patient to the and belate stress and by please prone and to the

-
Input sentence: Regrona Health inc	Regional Health Inc

Decoded sentence: Patient Name:

-
Input sentence: 	In-Network

Decoded sentence: Employee Poller Information

-
Input sentence: Out of Network 	Out of Network

Decoded sentence: Employee Poller Information

-
Input sentence: Claim Detail	Claim Detail

Decoded sentence: Call toll-free Monday through (mm/dd/yy)

-
Input sentence: oiaimii-	Claim#:

Decoded sentence: Patient Name:

-
Input sentence: ..Pat'ent	Patient:

Decoded sentence: Claim Date:

-
Input sentence: "lProvidei'E-KadAJLend	Provider: Karl A. Lund

Decoded sentence: Procedure Provider Information

-
Input sentence: Billed "Amount	Billed Amount

Decoded sentence: Call toll-free Monday through (mm/dd/yy)

-
Input sentence: Contract Adjuimenii	Contract Adjustment

Decoded sentence: Patient Name:

-
Input sentence: UCRlNon- Par Augustinian:	URC/Non-Par Adjustment

Decoded sentence: Employee Poller Information

-
Input sentence: Allowed Aﬂount	Allowed Amount

Decoded sent

...

-
Input sentence: iCD Code:	ICD Code:

Decoded sentence: Provider Signed Information

-
Input sentence: ii you. on what dais {mmftidiyyi’i’	If yes, on what date (mm/dd/yy)?

Decoded sentence: Provider Signed Information

-
Input sentence: If yas, plazas: uplain:	If yes, please explain:

Decoded sentence: Employee Poller Information

-
Input sentence: Data oi Saar affine visit (mmrddfyyi:	Date of last office visit (mm/dd/yy):

Decoded sentence: Cate Dischork of Sectide the to the patient to the and belate stress and by please prone and to the 
-
Input sentence: ate nf next ofﬁca visit (mmiddfw):	Date of next office visit (mm/dd/yy):

Decoded sentence: Cate Dischork of Sectide the to the patient to the and belate stress and by please prone and to the 
-
Input sentence: Did you advise your maﬁa"! in map working?	Did you advice your patient to stop working? Yes No

Decoded sentence: Provider Signed Information

-
Input sentence: [mmiddiyyiz We: iryeamnwnammeimmrddiyy)?	If yes, on what date

-
Input sentence: Status Bilied	Status Billed

Decoded sentence: Cate Dischork of Sectide the to the patient to the and belate stress and by please prone and to the 
-
Input sentence: Coverage	Coverage

Decoded sentence: Patient Name:

-
Input sentence: Guarantor Account {for Hospital Account }	Guarantor Account (for Hospital Account )

Decoded sentence: Country:

-
Input sentence: Name	Name

Decoded sentence: Patient Name:

-
Input sentence: Relation to 9t Sei‘f	Relation to Pt Self

Decoded sentence: Cate Dischork of Sectide the to the patient to the and belate stress and by please prone and to the 
-
Input sentence: Service Area SEH	Service Area SEH

Decoded sentence: Provider Signed Information

-
Input sentence: Active? Yes	Active? Yes

Decoded sentence: Employee Patient Name:

-
Input sentence: Sei‘f SEH Yes Personali'i-‘amrly	Acct Type Personal/Family

Decoded sentence: Patient Name:

-
Input sentence: Address	Address

Decoded sentence: Address Line 1:

-
Input sentence: Phone	Ph

-
Input sentence: Address Line 1:	Address Line 1:

Decoded sentence: Address Line 1:

-
Input sentence: City:	City:

Decoded sentence: Catrent Name:

-
Input sentence: Slater‘Pronnce:	State/Province

Decoded sentence: Confirmation - Health (mm/dd/yy)

-
Input sentence: Postal Code:	Postal Code:

Decoded sentence: Patient Name:

-
Input sentence: C mm’rry',	Country:

Decoded sentence: Employee Poller Information

-
Input sentence: “Fork State-Comm:	Work State/Country:

Decoded sentence: Current Name:

-
Input sentence: Best Phone Number to be Reached During the Day	Best Phone Number to be Reached During the Day

Decoded sentence: Call toll-free Monday through (mm/dd/yy)

-
Input sentence: Email Address:	Email Address:

Decoded sentence: Employee Poller Information

-
Input sentence: Spou s 0 Information	Spouse Information

Decoded sentence: Current Name:

-
Input sentence: First Name:	First Name:

Decoded sentence: Provider Signed Information

-
Input sentence: Middle Name/initial:	Midd

-
Input sentence: Claim Details	Claim Details

Decoded sentence: Call toll-free Monday through (mm/dd/yy)

-
Input sentence: 	Claim #

Decoded sentence: Call toll-free Monday through (mm/dd/yy)

-
Input sentence: Mamba: No:	Member No:

Decoded sentence: Patient Name:

-
Input sentence: Amount Billed:	Amount Billed

Decoded sentence: Employee Poller Information

-
Input sentence: Allowed Amount:	Allowed Amount:

Decoded sentence: Employee Poller Information

-
Input sentence: Paramount Paid: E 30039	Paramount Paid:

Decoded sentence: Patient Name:

-
Input sentence: Providen'Facility: ProMedicn Tole do Hospital	Provider/Facility: ProMedica Toledo Hospital

Decoded sentence: Provider Signed Information

-
Input sentence: 	Status PAYABLE

Decoded sentence: Cate Dischork of Sectide the to the patient to the and belate stress and by please prone and to the 
-
Input sentence: Service Details for This Clc-lm	Service Details for This Claim

Decoded sentence: Provider Signed Information

-
Inpu

-
Input sentence: StatefProvince :	State/Province:

Decoded sentence: Cate Dischork of Sectide the to the patient to the and belate stress and by please prone and to the 
-
Input sentence: Postal Code:	Postal Code:

Decoded sentence: Patient Name:

-
Input sentence: Country	Country:

Decoded sentence: Confirmation - Health (mm/dd/yy)

-
Input sentence: Best Phone Number to be Reached Dming the Day	Best Phone Number to be Reached During the Day

Decoded sentence: Call toll-free Monday through (mm/dd/yy)

-
Input sentence: Email Address:	Email Address:

Decoded sentence: Employee Poller Information

-
Input sentence: Daytime Phone:	Daytime Phone:

Decoded sentence: Cate Dischork of Sectide the to the patient to the and belate stress and by please prone and to the 
-
Input sentence: Dependent Information	Dependent Information

Decoded sentence: Country:

-
Input sentence: First Name:	First Name:

Decoded sentence: Provider Signed Information

-
Input sentence: Middle Nameﬂnitial:	Middle N